# CCA175 Practice Exam
* Exam    : CCA175
* Title   : CCA Spark and Hadoop Developer Exam
* Vendor  : Cloudera
* Version : V12.35

IT Certification Guaranteed, The Easy Way!

## NO.22 CORRECT TEXT
### Problem Scenario 31 : 

You have given following two files

1. Content.txt: Contain a huge text file containing space separated words.
2. Remove.txt: Ignore/filter all the words given in this file (Comma Separated).

Write a Spark program which reads the Content.txt file and load as an RDD, remove all the words
from a broadcast variables (which is loaded as an RDD of words from Remove.txt).
And count the occurrence of the each word and save it as a text file in HDFS.

Content.txt

In [4]:
Hello this is ABCTech.com
This is TechABY.com
Apache Spark Training
This is Spark Learning Session
Spark is faster than MapReduce

Remove.txt

In [6]:
Hello, is, this, the

#### Answer

In [8]:
See the explanation for Step by Step Solution and configuration.
Explanation:
Solution (in scala):

Step 1 : Create all three files in hdfs in directory called spark2 (We will do using Hue).
However, you can first create in local filesystem and then upload it to hdfs

Step 2 : Load the Content.txt file
val content = sc.textFile("spark2/Content.txt") //Load the text file

Step 3 : Load the Remove.txt file
val remove = sc.textFile("spark2/Remove.txt") //Load the text file

Step 4 : Create an RDD from remove, However, there is a possibility each word could have trailing
spaces, remove those whitespaces as well. We have used two functions here flatMap, map and trim.
val removeRDD = remove.flatMap( x=> x.split( ",") ).map(word=>word.trim)//Create an array of words

Step 5 : Broadcast the variable, which you want to ignore
val bRemove = sc.broadcast(removeRDD.collect().toList) // It should be array of Strings
                              
Step 6 : Split the content RDD, so we can have Array of String. 
val words = content.flatMap(line => line.split(" "))
                              
Step 7 : Filter the RDD, so it can have only content which are not present in "Broadcast
Variable". 
val filtered = words.filter( case (word) => !bRemove.value.contains(word) )

Step 8 : Create a PairRDD, so we can have (word,1) tuple or PairRDD. 
val pairRDD = filtered.map( word => (word,1) )

Step 9 : Nowdo the word count on PairRDD. val wordCount = pairRDD.reduceByKey(_ + _)

Step 10 : Save the output as a Text file.
wordCount.saveAsTextFile("spark2/result.txt")
                              

solution in pyspark

In [10]:
content_rdd = sc.textFile( 'spark2/Content.txt' )
            .flatMap( lambda line: line.split( ' ' ) )
            .map    ( lambda s: s.strip() )
            .filter ( lambda s: s!= None and s!= '' and s!= ' ' )

remove_rdd  = sc.textFile( 'spark2/Remove.txt'  )
            .flatMap( lambda line: line.split( ',' ) )
            .map    ( lambda s: s.strip() )
            .filter ( lambda s: s!= None and s!= '' and s!= ' ' )

b_remove = sc.broadcast( remove_rdd.collect() )

In [11]:
filtered = content_rdd.filter( lambda word: word not in b_remove.value )

In [12]:
# count words. 
word_count = filtered.map( lambda word: (word, 1)  ).reduceByKey( lambda v1, v2: v1 + v2 )

# count words and sort, most common word t top.
word_count = filtered.map( lambda word: (word, 1)  ).reduceByKey( lambda v1, v2: v1 + v2 ).map( lambda (w,n): (n, w) ).sortByKey( False)

In [13]:
word_count.saveAsTextFile( 'spark2/result.txt' )

end
---